# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math
import collections


def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


def hsv(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)


def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

LaneLine = collections.namedtuple('LaneLine', ['slope', 'x', 'y'])


def draw_lines(img, lines, slope_threshold=0.1, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    global prev_pos_slopes
    global prev_neg_slopes
            
    pos_slopes = []
    neg_slopes = []
    for line in lines:
        #print('line = ', line)
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/ (x2 - x1))
            if (abs(slope) > slope_threshold):
                middle_x = ((x2 + x1) * 0.5)
                middle_y = ((y2 + y1) * 0.5)
                ll = LaneLine(slope, middle_x, middle_y)
                #print('slope = ', getattr(ll, 'slope'))
                pos_slopes.append(ll) if (slope > 0) else neg_slopes.append(ll)
                #cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    #return

    np_n_slopes = np.array(list(getattr(ll, 'slope') for ll in neg_slopes))
    np_p_slopes = np.array(list(getattr(ll, 'slope') for ll in pos_slopes))
    neg_std = np.std(np_n_slopes)
    pos_std = np.std(np_p_slopes)
    neg_median = np.median(np_n_slopes)
    pos_median = np.median(np_p_slopes)
    #print('neg std dev = ', neg_std, 'pos std dev = ', pos_std)
    #print('neg median = ', neg_median, 'pos median = ', pos_median)

    img_y = img.shape[0]
    img_x = img.shape[1]
    
    weights = [0.4, 0.3, 0.2, 0.1]
    for slopes, prev, median, stddev in [(neg_slopes, prev_neg_slopes, neg_median, neg_std), \
                                         (pos_slopes, prev_pos_slopes, pos_median, pos_std)]:
        valid_slopes = list((ll for ll in slopes if abs(getattr(ll, 'slope') - median) < 1 * stddev))
        #print('valid_slopes = ', len(valid_slopes))
        if len(valid_slopes):
            avg_slope = sum(list(getattr(ll, 'slope') for ll in valid_slopes)) / float(len(valid_slopes))
            avg_x = sum(list(getattr(ll, 'x') for ll in valid_slopes)) / float(len(valid_slopes))
            avg_y = sum(list(getattr(ll, 'y') for ll in valid_slopes)) / float(len(valid_slopes))
            #print('avg_slope = ', avg_slope, ' x = ', avg_x, ' y = ', avg_y)
            if len(prev):
                prev.pop(0)
            while len(prev) < len(weights):
                prev.append(LaneLine(avg_slope, avg_x, avg_y))
        if len(prev):
            cur_slope = sum(list(weights[i] * getattr(prev[i], 'slope') for i in range(len(weights))))
            cur_x = sum(list(weights[i] * getattr(prev[i], 'x') for i in range(len(weights))))
            cur_y = sum(list(weights[i] * getattr(prev[i], 'y') for i in range(len(weights))))
            prev[-1] = LaneLine(cur_slope, cur_x, cur_y)
            pos_y1 = img_y
            pos_x1 = int(((pos_y1 - cur_y) / cur_slope) + cur_x)
            pos_y2 = int(img_y * 0.65) # from region of interest
            pos_x2 = int(((pos_y2 - cur_y) / cur_slope) + cur_x)
            cv2.line(img, (pos_x1, pos_y1), (pos_x2, pos_y2), color, thickness)

    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines, slope_threshold=0.2, thickness=10)
    return line_img

# Python 3 has support for cool math symbols.


def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def apply_hsv_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    w_lower = np.array([0, 0, 210])
    w_upper = np.array([180, 255, 255])
    mask_w = cv2.inRange(hsv, w_lower, w_upper)

    masked = cv2.bitwise_and(image, image, mask=mask_w)
    return masked
    
    
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,blu
    # you should return the final output (image with lines are drawn on lanes
    
    # Defining vertices for region of interest
    x_size = image.shape[1]
    y_size = image.shape[0]
    vertices = np.array([[(x_size * 0.1, y_size), (x_size * 0.45, y_size * 0.6), \
                          (x_size * 0.55, y_size * 0.6), (x_size * 0.9, y_size)]], dtype=np.int32)

    trim_image = region_of_interest(image, vertices)
    color_sel = apply_hsv_mask(trim_image)

    gray = grayscale(color_sel)
    gray = cv2.equalizeHist(gray)
    blur_gray = gaussian_blur(gray, kernel_size=5)
    edges = canny(blur_gray, low_threshold=50, high_threshold=150)
    masked_edges = region_of_interest(edges, vertices)

    line_image = hough_lines(masked_edges, rho=1, theta = np.pi/180,
                             threshold=10, min_line_len=10, max_line_gap=5)
    result = weighted_img(line_image, image, α=0.8, β=1., λ=0.)
    return result

# DEBUG
image_list = os.listdir("test_images/")
for img_path in image_list:
    prev_neg_slopes = []
    prev_pos_slopes = []
    image = mpimg.imread("test_images/%s" % img_path)
    plt.figure()
    plt.imshow(image)
    plt.figure()
    plt.imshow(process_image(image))

Let's try the one with the solid white lane on the right first ...

In [ ]:
prev_neg_slopes = []
prev_pos_slopes = []
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
prev_neg_slopes = []
prev_pos_slopes = []
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

## Improvements

There are several possible enhancements for this algorithm - it is particularly not robust under challenging light conditions and where constrat between lane markers and the road is poor. It is also very likely to fail if we were to find painted markings in the road (crosswalks, stop and crossing signs, etc).

Below a few ideas to improve dealing with light conditions and contrast:

### Use lane markers from previous frames (implemented)

Since lane markers shouldn't abruptly change direction, it is possible to use lane markers detected in previous frames when calculating the lane markers for the current frame. This provides very good hints under tough conditions, keep the markers transitioning smoothly and deal with possible undetected markers.

### Adpative threshols for color selection (implemented and discarded)

Use an adaptive threshold for color selection. Since we might end up masking too much or too little information from the image, a possibility is to adjust the masking thresholds based on a minimum percentage of pixels that must be non-black during color selection. The obvious drawback is the extra computational effort required to perform multiple color selection runs per frame. (tested and removed given the extra computational effort compared to additional gain)

### Filter outliers (implemented)

Use slopes median and standard deviation to discard outliers.

### Filter lines with unexpected slopes (implemented)

Given the position of the camera in the car, we can assume (when already inside a lane) that the lane markers slopes would fall in a specified range. Although this works well for the kind of scenarios in this lesson, it would probably not work when dealing with other common situation (e.g., a conversion in an intersection where the lanes we're looking for are perpendicular to the car).

### Dealing with missing markers

If one lane marker is missing (alg. couldn't detect it) we can still use the opposite lane marker as a guide for where we would expect the missing lane marker to be. Not very important since this is also addressed by the weighted average of markers found on previous frames.

### Improving slope accuracy

When calculating the slope, give a higher weight to points closer to the camera, since those tend to be more accurate.

### Use more than one color space for color selection (partially implemented)

Use other color spaces in order to perform a more robust color selection. 

### Other image improvements (partially implemented)

In the implemented algorithm we're equalizing the image to improve contrast. Better constrast improvement algorithms could be used to more robustly detect the lane markers.


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
prev_neg_slopes = []
prev_pos_slopes = []
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))